<a href="https://colab.research.google.com/github/doubi-xu/Political-Ideology-and-Customer-Feedback-Behaviors/blob/main/Feedback_DiagnosticValue_Study4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### STEP 0: Install Dependencies ###
!pip install pandas pyreadstat bertopic umap-learn hdbscan nltk -q
!pip install sentence-transformers -q

### STEP 1: Mount Google Drive ###
from google.colab import drive
drive.mount('/content/drive')

### STEP 2: Load Dataset ###
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

file_path = '/content/drive/My Drive/ACSIFeedback/cars_reviewlevel04122025.dta'
df = pd.read_stata(file_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.7/617.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.6 MB/s eta 0:00:00
Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
### STEP 3: Stratified Sampling by Review Rating ###
rating_dist = df['reviewrating'].value_counts(normalize=True).sort_index()
sample_sizes = (rating_dist * 50000).round().astype(int)

sampled = []
for rating, n in sample_sizes.items():
    group = df[df['reviewrating'] == rating]
    available_n = len(group)
    if available_n > 0 and n > 0:
        n_sample = min(n, available_n)
        sampled.append(group.sample(n=n_sample, random_state=42))

sample_df = pd.concat(sampled).reset_index(drop=True)
print(f"✅ Sampled {len(sample_df)} reviews.")

# Save the sample to Drive
sample_path = "/content/drive/My Drive/ACSIFeedback/cars_sample50000.csv"
sample_df.to_csv(sample_path, index=False)
print(f"💾 Saved sampled reviews to: {sample_path}")

### STEP 4: Prepare Texts for BERTopic ###
sample_df['reviewtext_cleaned'] = sample_df['reviewtext'].fillna("")
texts_cleaned = sample_df['reviewtext_cleaned'].tolist()

### STEP 5: Topic Modeling with BERTopic ###
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
topic_model = BERTopic(embedding_model=embedding_model, calculate_probabilities=True, verbose=True)

topics, probs = topic_model.fit_transform(texts_cleaned)

# Save model and outputs
topic_model.save("/content/drive/My Drive/ACSIFeedback/bertopic_model_cars50000")
doc_info = topic_model.get_document_info(texts_cleaned)
doc_info.to_csv("/content/drive/My Drive/ACSIFeedback/topic_docinfo_cars50000.csv", index=False)
topic_info = topic_model.get_topic_info()
topic_info.to_csv("/content/drive/My Drive/ACSIFeedback/topic_overview_cars50000.csv", index=False)

print("📄 Topic model and outputs saved successfully.")


✅ Sampled 49999 reviews.
💾 Saved sampled reviews to: /content/drive/My Drive/ACSIFeedback/cars_sample50000.csv


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-06-13 01:07:50,628 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

2025-06-13 01:08:08,982 - BERTopic - Embedding - Completed ✓
2025-06-13 01:08:08,983 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-13 01:08:45,908 - BERTopic - Dimensionality - Completed ✓
2025-06-13 01:08:45,909 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-13 01:17:53,416 - BERTopic - Cluster - Completed ✓
2025-06-13 01:17:53,430 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-13 01:17:55,580 - BERTopic - Representation - Completed ✓
2025-06-13 01:17:57,548 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


📄 Topic model and outputs saved successfully.


In [ ]:
### STEP delete: Stratified Sampling by Review Rating – 10 000 additional ###

# 1. Compute proportional targets for 10 000 rows
rating_dist = df['reviewrating'].value_counts(normalize=True).sort_index()
sample_sizes2 = (rating_dist * 10000).round().astype(int)

# 2. Draw the sample
sampled2 = []
for rating, n in sample_sizes2.items():
    group = df[df['reviewrating'] == rating]
    if n > 0 and len(group) > 0:
        n_sample = min(n, len(group))
        sampled2.append(group.sample(n=n_sample, random_state=123, replace=False))

sample_df2 = pd.concat(sampled2).reset_index(drop=True)
print(f"✅ Sampled {len(sample_df2)} reviews for second sample.")

# 3. Save to Drive
sample_path2 = "/content/drive/My Drive/ACSIFeedback/cars_sample10000.csv"
sample_df2.to_csv(sample_path2, index=False)
print(f"💾 Saved second sample to: {sample_path2}")

✅ Sampled 9999 reviews for second sample.
💾 Saved second sample to: /content/drive/My Drive/ACSIFeedback/cars_sample10000.csv


In [ ]:
### STEP 6: Reload Full Dataset and BERTopic Model ###
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


# Reload original full dataset
file_path = "/content/drive/My Drive/ACSIFeedback/cars_sample10000.csv"
df_full = pd.read_csv(file_path)
df_full['reviewtext_cleaned'] = df_full['reviewtext'].fillna("")

# Reload saved model
model_path = "/content/drive/My Drive/ACSIFeedback/bertopic_model_cars50000"
topic_model = BERTopic.load(model_path)

# Apply to full dataset
texts_full = df_full['reviewtext_cleaned'].tolist()
topics_full, probs_full = topic_model.transform(texts_full)

print("✅ Topic model applied to full dataset.")



Batches:   0%|          | 0/313 [00:00<?, ?it/s]

2025-06-18 14:08:06,922 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-06-18 14:08:16,523 - BERTopic - Dimensionality - Completed ✓
2025-06-18 14:08:16,523 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-06-18 14:08:17,968 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2025-06-18 14:10:12,571 - BERTopic - Probabilities - Completed ✓
2025-06-18 14:10:12,572 - BERTopic - Cluster - Completed ✓


✅ Topic model applied to full dataset.


In [ ]:
### STEP 7: Load Formative Topics ###
import pandas as pd

# Load manually coded topics
coded_path = "/content/drive/My Drive/ACSIFeedback/topic_overview_cars50000_coded.csv"
coded_df = pd.read_csv(coded_path)

# Extract list of formative topic IDs
formative_topics = coded_df[coded_df["Formative"] == 1]["Topic"].tolist()
print(f"📌 {len(formative_topics)} formative topics loaded: {formative_topics[:10]}...")

📌 208 formative topics loaded: [-1, 0, 1, 2, 3, 4, 9, 10, 14, 16]...


In [ ]:


### STEP 8: Compute Diagnostic Value for Each Review ###
# Compute review length
df_full['review_length'] = df_full['reviewtext_cleaned'].apply(lambda x: len(x.split()))
avg_length = df_full['review_length'].mean()

# Add topic probabilities as columns
for i in formative_topics:
    df_full[f'Topic_{i}'] = [p[i] if p is not None and i < len(p) else 0 for p in probs_full]

# Calculate diagnostic score (scaled by relative length)
def calc_diagnostic_adjusted(row, formative_ids, avg_len):
    topic_sum = sum([row[f'Topic_{i}'] for i in formative_ids if f'Topic_{i}' in row])
    return topic_sum * (row['review_length'] / avg_len)

df_full['diagnostic_value'] = df_full.apply(lambda row: calc_diagnostic_adjusted(row, formative_topics, avg_length), axis=1)

print("✅ Diagnostic value calculated.")

<ipython-input-11-1540884255>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f'Topic_{i}'] = [p[i] if p is not None and i < len(p) else 0 for p in probs_full]
<ipython-input-11-1540884255>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f'Topic_{i}'] = [p[i] if p is not None and i < len(p) else 0 for p in probs_full]
<ipython-input-11-1540884255>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining

✅ Diagnostic value calculated.


<ipython-input-11-1540884255>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full['diagnostic_value'] = df_full.apply(lambda row: calc_diagnostic_adjusted(row, formative_topics, avg_length), axis=1)


In [ ]:


### STEP 9: Save Final Annotated Output ###
output_path = '/content/drive/My Drive/ACSIFeedback/full_reviews_with_diagnostic_value_cars10000.csv'
df_full.to_csv(output_path, index=False)
print(f"💾 Saved final dataset to: {output_path}")


💾 Saved final dataset to: /content/drive/My Drive/ACSIFeedback/full_reviews_with_diagnostic_value_cars10000.csv
